In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
from collections import OrderedDict

In [2]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor())

/nfshome/lampt/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1631630815121/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor())

In [4]:
BATCH_SIZE = 64
train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [5]:
for X, y in train_dataloader:
    print(X.shape)
    print(y, y.shape, y.dtype)
    break

torch.Size([64, 1, 28, 28])
tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9, 1, 0, 6, 4, 3, 1, 4, 8,
        4, 3, 0, 2, 4, 4, 5, 3, 6, 6, 0, 8, 5, 2, 1, 6, 6, 7, 9, 5, 9, 2, 7, 3,
        0, 3, 3, 3, 7, 2, 2, 6, 6, 8, 3, 3, 5, 0, 5, 5]) torch.Size([64]) torch.int64


In [6]:
len(train_dataloader)

938

In [7]:
len(test_dataloader)

157

In [8]:
train_dataloader.dataset

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: {}'.format(device))

Device: cpu


In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            OrderedDict([
            ('fc1', nn.Linear(28*28, 512)),
            ('relu1', nn.ReLU()),
            ('fc2', nn.Linear(512, 512)),
            ('relu2', nn.ReLU()),
            ('fc3', nn.Linear(512, 10))
            ])
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (fc1): Linear(in_features=784, out_features=512, bias=True)
    (relu1): ReLU()
    (fc2): Linear(in_features=512, out_features=512, bias=True)
    (relu2): ReLU()
    (fc3): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
for name, weight in list(model.named_parameters()):
    print(name,': ', weight.shape)

linear_relu_stack.fc1.weight :  torch.Size([512, 784])
linear_relu_stack.fc1.bias :  torch.Size([512])
linear_relu_stack.fc2.weight :  torch.Size([512, 512])
linear_relu_stack.fc2.bias :  torch.Size([512])
linear_relu_stack.fc3.weight :  torch.Size([10, 512])
linear_relu_stack.fc3.bias :  torch.Size([10])


In [50]:
list(model.named_parameters())

[('linear_relu_stack.fc1.weight',
  Parameter containing:
  tensor([[ 0.0305, -0.0352,  0.0105,  ..., -0.0027, -0.0287, -0.0143],
          [ 0.0117, -0.0267, -0.0348,  ..., -0.0075,  0.0190, -0.0100],
          [-0.0247,  0.0282,  0.0133,  ...,  0.0104, -0.0223,  0.0199],
          ...,
          [ 0.0205,  0.0046, -0.0286,  ...,  0.0322, -0.0049, -0.0305],
          [ 0.0019,  0.0037,  0.0115,  ..., -0.0068, -0.0037,  0.0300],
          [-0.0002, -0.0036, -0.0349,  ...,  0.0347, -0.0303, -0.0347]],
         requires_grad=True)),
 ('linear_relu_stack.fc1.bias',
  Parameter containing:
  tensor([-0.0106,  0.0294,  0.0227, -0.0094, -0.0031,  0.0194, -0.0161,  0.0257,
          -0.0192, -0.0057,  0.0116,  0.0353,  0.0334,  0.0236,  0.0057,  0.0044,
           0.0108,  0.0052, -0.0134, -0.0078,  0.0065,  0.0245,  0.0310,  0.0139,
           0.0076, -0.0220, -0.0251, -0.0086,  0.0013,  0.0236,  0.0290,  0.0017,
           0.0010,  0.0058,  0.0198,  0.0013, -0.0225,  0.0072, -0.0061, -0.014

In [23]:
inputs = torch.zeros([32, 28*28])

In [29]:
model(inputs).shape

torch.Size([32, 10])

In [41]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [13]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
#     model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Back propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss, batch*len(X)
            print(f'Loss: {loss:>7f}   [{current:>5d}/{size:>5d}]')

In [44]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y)
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [20]:
test_input, test_groundtruth = list(train_dataloader)[0]

In [27]:
test_pred = model(test_input)

In [39]:
(test_pred.argmax(1)==test_groundtruth).type(torch.float).sum().item()

5.0

In [32]:
test_groundtruth

tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9, 1, 0, 6, 4, 3, 1, 4, 8,
        4, 3, 0, 2, 4, 4, 5, 3, 6, 6, 0, 8, 5, 2, 1, 6, 6, 7, 9, 5, 9, 2, 7, 3,
        0, 3, 3, 3, 7, 2, 2, 6, 6, 8, 3, 3, 5, 0, 5, 5])

In [45]:
EPOCHS = 5
for t in range(EPOCHS):
    print(f'EPOCH: {t+1}\n--------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print('Done!!!')

EPOCH: 1
--------------------
Loss: 1.899814   [    0/60000]
Loss: 1.859255   [ 6400/60000]
Loss: 1.753117   [12800/60000]
Loss: 1.787033   [19200/60000]
Loss: 1.656033   [25600/60000]
Loss: 1.622410   [32000/60000]
Loss: 1.635193   [38400/60000]
Loss: 1.545826   [44800/60000]
Loss: 1.580257   [51200/60000]
Loss: 1.459413   [57600/60000]
Test Error: 
 Accuracy: 59.7%, Avg loss: 1.487367 

EPOCH: 2
--------------------
Loss: 1.561848   [    0/60000]
Loss: 1.515023   [ 6400/60000]
Loss: 1.380384   [12800/60000]
Loss: 1.447077   [19200/60000]
Loss: 1.319578   [25600/60000]
Loss: 1.328356   [32000/60000]
Loss: 1.335279   [38400/60000]
Loss: 1.265890   [44800/60000]
Loss: 1.310010   [51200/60000]
Loss: 1.202843   [57600/60000]
Test Error: 
 Accuracy: 63.0%, Avg loss: 1.232235 

EPOCH: 3
--------------------
Loss: 1.310945   [    0/60000]
Loss: 1.282459   [ 6400/60000]
Loss: 1.131635   [12800/60000]
Loss: 1.236456   [19200/60000]
Loss: 1.110289   [25600/60000]
Loss: 1.138770   [32000/60000]


In [54]:
model.state_dict()

OrderedDict([('linear_relu_stack.fc1.weight',
              tensor([[-0.0168, -0.0298,  0.0188,  ..., -0.0142, -0.0108, -0.0236],
                      [-0.0329,  0.0217, -0.0120,  ..., -0.0172, -0.0308, -0.0194],
                      [ 0.0170, -0.0076,  0.0331,  ...,  0.0356, -0.0102,  0.0183],
                      ...,
                      [-0.0037, -0.0304, -0.0345,  ...,  0.0271, -0.0267, -0.0107],
                      [-0.0139, -0.0170,  0.0017,  ..., -0.0048, -0.0167, -0.0114],
                      [ 0.0130, -0.0021,  0.0176,  ...,  0.0187,  0.0328,  0.0239]])),
             ('linear_relu_stack.fc1.bias',
              tensor([-0.0147,  0.0221,  0.0341, -0.0286, -0.0005, -0.0139,  0.0376,  0.0349,
                       0.0010,  0.0295, -0.0295,  0.0093,  0.0326,  0.0213,  0.0070,  0.0275,
                       0.0018, -0.0213,  0.0279, -0.0082,  0.0225, -0.0038, -0.0098,  0.0283,
                       0.0008,  0.0296,  0.0158,  0.0028,  0.0323,  0.0188,  0.0107,  0.0088,


In [55]:
torch.save(model.state_dict(), 'model.pth')

In [59]:
train_dataloader[0]

TypeError: 'DataLoader' object is not subscriptable

In [ ]:
torch.load('mo')

In [64]:
new_model = NeuralNetwork()
new_model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [60]:
!ls -la

total 3162
drwxrwxr-x 2 lampt lampt    4096 17 Th10  2021 .
drwx------ 2 lampt lampt    4096 11:56 16 Th10 ..
-rw-rw-r-- 1 lampt lampt   48960 17 Th10  2021 A_simple_workflow.ipynb
drwxrwxr-x 2 lampt lampt      64 10:06 17 Th10 data
drwxrwxr-x 2 lampt lampt    4096 11:56 16 Th10 .git
drwxrwxr-x 2 lampt lampt      64 17 Th10  2021 .ipynb_checkpoints
-rw-rw-r-- 1 lampt lampt 2680919 17 Th10  2021 model.pth
-rw-rw-r-- 1 lampt lampt     187 11:56 16 Th10 README.md
